In [3]:
import streamlit as st
import pandas as pd
import requests
import plotly.graph_objects as go
from datetime import datetime

# Set your Polygon.io API key
API_KEY = 'Sbneu6PsHV64vNjEumFu7YYabFb7BBQJ'

def get_real_time_price(symbol):
    url = f'https://api.polygon.io/v2/aggs/ticker/{symbol}/prev?adjusted=true&apiKey={API_KEY}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'Error': f"API request failed with status code: {response.status_code}"}

def get_historical_price(symbol, from_date, to_date, interval):
    url = f"https://api.polygon.io/v2/aggs/ticker/{symbol}/range/{interval}/minute/{from_date}/{to_date}?adjusted=true&sort=asc&limit=5000&apiKey={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return {'Error': f"API request failed with status code: {response.status_code}"}

def compute_rsi(df, period=14):
    """Calculate RSI indicator from close prices"""
    delta = df['c'].diff()
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def display_stock_price(symbol, interval):
    interval_map = {
        '1 minute': '1',
        '5 minute': '5',
        '15 minute': '15',
        '30 minute': '30',
        '1 hour': '60'
    }

    try:
        if interval == 'Real-time':
            data = get_real_time_price(symbol)
            if 'Error' in data:
                st.error(data['Error'])
                return
            df = pd.DataFrame([data['results'][0]])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df.set_index('t', inplace=True)
            st.write(f"**{symbol} (as of {df.index[-1]})**")

            st.warning("RSI not available for real-time view. Please select historical interval.")
        else:
            from_date = st.date_input("From Date:", value=pd.Timestamp(year=2024, month=1, day=1))
            to_date = st.date_input("To Date:", value=datetime.today())

            data = get_historical_price(symbol, from_date.strftime('%Y-%m-%d'), to_date.strftime('%Y-%m-%d'), interval_map[interval])
            if 'Error' in data:
                st.error(data['Error'])
                return
            if 'results' not in data or not data['results']:
                st.warning("No data available for the selected range.")
                return

            df = pd.DataFrame(data['results'])
            df['t'] = pd.to_datetime(df['t'], unit='ms')
            df.set_index('t', inplace=True)

            # Calculate RSI
            df['RSI'] = compute_rsi(df)

            # Candlestick Chart
            st.write("### OHLC Candlestick Chart")
            fig1 = go.Figure(data=[go.Candlestick(
                x=df.index,
                open=df['o'],
                high=df['h'],
                low=df['l'],
                close=df['c']
            )])
            fig1.update_layout(title=f"{symbol} Candlestick Chart",
                              xaxis_title='Date',
                              yaxis_title='Price',
                              xaxis_rangeslider_visible=False)
            st.plotly_chart(fig1, use_container_width=True)

            # RSI Chart
            st.write("### Relative Strength Index (RSI)")
            fig2 = go.Figure()
            fig2.add_trace(go.Scatter(x=df.index, y=df['RSI'], mode='lines', name='RSI', line=dict(color='blue')))
            fig2.add_hline(y=70, line_dash="dot", line_color="red", annotation_text="Overbought", annotation_position="top left")
            fig2.add_hline(y=30, line_dash="dot", line_color="green", annotation_text="Oversold", annotation_position="bottom left")
            fig2.update_layout(title="RSI Indicator (14-period)", yaxis_title='RSI Value', xaxis_title='Date')
            st.plotly_chart(fig2, use_container_width=True)

    except requests.exceptions.RequestException as e:
        st.error("Connection error. Please check your internet.")
    except Exception as e:
        st.error(f"An unexpected error occurred: {str(e)}")

def main():
    st.title("📈 Stock Dashboard with RSI")
    symbol = st.text_input("Enter Stock Symbol (e.g., AAPL, MSFT, TSLA):").upper()
    interval_options = ["Real-time", "1 minute", "5 minute", "15 minute", "30 minute", "1 hour"]
    interval = st.selectbox("Select Time Interval:", interval_options)

    if symbol:
        with st.spinner('Fetching stock data...'):
            display_stock_price(symbol, interval)

if __name__ == '__main__':
    main()


2025-06-14 15:10:16.780 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 15:10:17.028 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-06-14 15:10:17.029 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 15:10:17.032 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 15:10:17.037 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 15:10:17.038 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 15:10:17.039 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-06-14 15:10:17.041 Session state does not 